In [1]:
base = "evaluation/decompile_result_base.json"
test_pair = "evaluation/decompile_result_test_pair.json"
orig = "decompile-eval/decompile-eval-executable-gcc-ghidra.json"

In [2]:
import json

with open(base, "r") as f:
    base = json.load(f)

with open(test_pair, "r") as f:
    test_pair = json.load(f)
with open(orig, "r") as f:
    orig = json.load(f)

In [3]:
import re

In [12]:
def extract_assertions(func):
    # Pattern to capture func0 call within an assert statement
    pattern = re.compile(r"assert\s*\(\s*func0\s*\(([^)]+)\)\s*==\s*(\d+)\s*\);")

    # Find all assertions and store as (input_args_list, output_value) tuples
    results = []
    matches = pattern.findall(func)
    for match in matches:
        # Split function arguments and strip whitespace
        args = match[0].split(",")
        input_args_list = [arg.strip() for arg in args]

        # Expected output is the second captured group
        output_value = int(match[1])

        # Append result as a tuple
        results.append((input_args_list, output_value))

    return results

In [4]:
def extract_assertions_with_type_check(code):
    # Patterns to capture array definitions and assert statements
    array_pattern = re.compile(r"float\s+(\w+)\s*\[\]\s*=\s*\{([^}]+)\};")
    assert_pattern = re.compile(r"assert\s*\(\s*func0\s*\(([^)]+)\)\s*==\s*(\d+)\s*\);")

    # Dictionary to hold arrays by name
    arrays = {}

    # Find all array definitions
    for array_match in array_pattern.findall(code):
        array_name = array_match[0]
        # Split and convert the values inside the braces to floats
        array_values = [float(value.strip()) for value in array_match[1].split(",")]
        arrays[array_name] = array_values

    # Find all assertions and store as (input_args_list, output_value) tuples
    results = []
    for assert_match in assert_pattern.findall(code):
        # Extract function arguments and initialize the input argument list
        args = assert_match[0].split(",")
        input_args_list = []

        # Check each argument's type: if it's an array, replace it with its values
        for arg in args:
            arg = arg.strip()
            if arg in arrays:  # Check if the argument is a defined array variable
                input_args_list.append(arrays[arg])
            else:
                # Attempt to parse the argument as a literal (int or float)
                try:
                    if "." in arg:
                        input_args_list.append(float(arg))
                    else:
                        input_args_list.append(int(arg))
                except ValueError:
                    # In case of unexpected format, add it as a string
                    input_args_list.append(arg)

        # Expected output as integer
        output_value = int(assert_match[1])

        # Append the input arguments and output as a tuple
        results.append((input_args_list, output_value))

    return results

In [21]:
i=150

In [22]:
print(extract_assertions_with_type_check(orig[i]["c_test"]))

[]


In [23]:
print(extract_assertions(orig[i]["c_test"]))

[]


In [24]:
print(orig[i]["type"])

O2


In [25]:
print(orig[i]["c_test"])

#include <stdio.h>
#include <assert.h>
#include <math.h>


int issame(float *a, float *b, int size) {
    for (int i = 0; i < size; i++) {
        if (fabs(a[i] - b[i]) > 1e-4) {
            return 0;
        }
    }
    return 1;
}

int main() {
    float test1[] = {1, 2, 3};
    float result1[3];
    func0(test1, 3, result1);
    float expected1[] = {1, 2, 3};
    assert(issame(result1, expected1, 3));

    float test2[] = {5, 3, -5, 2, -3, 3, 9, 0, 123, 1, -10};
    float result2[11];
    func0(test2, 11, result2);
    float expected2[] = {-10, 3, -5, 2, -3, 3, 5, 0, 9, 1, 123};
    assert(issame(result2, expected2, 11));

    float test3[] = {5, 8, -12, 4, 23, 2, 3, 11, 12, -10};
    float result3[10];
    func0(test3, 10, result3);
    float expected3[] = {-12, 8, 3, 4, 5, 2, 12, 11, 23, -10};
    assert(issame(result3, expected3, 10));

    return 0;
}


In [20]:
print(orig[i]["c_func"])

#include <stdio.h>
#include <string.h>
#include <stdlib.h>

void func0(char *s, int encode) {
    int shift = encode ? 5 : 21;
    size_t len = strlen(s);
    for (size_t i = 0; i < len; i++) {
        int w = ((s[i] - 'a' + shift) % 26) + 'a';
        s[i] = (char)w;
    }
}


In [10]:
print(base[i])
print(test_pair[i])

int func0(int param_1, int *param_2) {
  int iVar1;
  int *pvVar2 = malloc(0x100);
  *param_2 = 0;
  int local_1c = param_1;
  int local_14;
  
  for (local_14 = 2; local_14 * local_14 <= local_1c; local_14++) {
    while (local_1c % local_14 == 0) {
      local_1c /= local_14;
      iVar1 = *param_2;
      *param_2 = iVar1 + 1;
      *(int *)((long)iVar1 * 4 + (long)pvVar2) = local_14;
    }
  }
  if (local_1c > 1) {
    iVar1 = *param_2;
    *param_2 = iVar1 + 1;
    *(int *)((long)iVar1 * 4 + (long)pvVar2) = local_1c;
  }
  return pvVar2;
}
// Refined source code from given pseudo code
#include <stdlib.h>

int *func0(int n, int *prime_count) {
    int *primes = (int *)malloc(n * sizeof(int));
    *prime_count = 0;
    int i = 2;
    while (i * i <= n) {
        while (n % i == 0) {
            n /= i;
            (*prime_count)++;
            primes[*prime_count - 1] = i;
        }
        i++;
    }
    if (n > 1) {
        (*prime_count)++;
        primes[*prime_count - 1] = n;
  

In [9]:
for b, t in zip(base, test_pair):
    print(b)
    print("=====")
    print(t)
    
    break

int func0(float param_1, long param_2, int param_3) {
  int local_10 = 0;
  while (param_3 > local_10) {
    int local_c = local_10;
    if (param_3 <= local_10) {
      return 0;
    }
    while (local_c < param_3) {
      if ((float)(*(float *)(param_2 + local_10 * 4) - *(float *)(param_2 + local_c * 4)) < param_1) {
        return 1;
      }
      local_c++;
    }
    local_10++;
  }
  return 0;
}
=====
float func0(char *param_1,long param_2,float param_3)

{
  int i;
  float diff;
  
  for (i = 0; i < strlen(param_1); i++) {
    diff = *(float *)(param_2 + (long)i * 4) - *(float *)(param_2 + (long)(i + 1) * 4);
    if (diff < param_3) {
      return 1;
    }
  }
  return 0;
}
